In [54]:
#Imports
import pandas as pd
from datetime import datetime
from g2p_en import G2p
from pyphonetics import Soundex
#pip install g2p_en <- erforderlich für die Phonemtranskribierung
#oder python setup.py install

In [55]:
#CSV einlesen und Dataframes erstellen
cols_twistlist = ['text', 'phonemes', 'keywords', 'source']
twistList = pd.read_csv(r'data source 2\raw-data\raw-data\twister_dataset.csv', delimiter=',',nrows= 3000,header=None, names=cols_twistlist)
cols_kaggle = ['0', 'text']
kaggle = pd.read_csv(r'data source 2\raw-data\raw-data\datasetKaggle.csv', delimiter=',',nrows= 1000,header=None, usecols=['text'], names=cols_kaggle)
kaggle = kaggle.drop(0)
twistList = twistList.drop(0)

In [56]:
twistList

,text,phonemes,keywords,source
1,Peter Piper picked a peck of pickled peppers. ...,[P IY1 T ER0] [P AY1 P ER0] [P IH1 K T] [AH0] ...,pickled peppers peter piper picked,https://www.kaggle.com/datasets/djathidiro/ton...
2,I saw Susie sitting in a shoe shine shop. Wher...,[AY1] [S AO1] [S UW1 Z IY0] [S IH1 T IH0 NG] [...,shoe shine shop,https://www.kaggle.com/datasets/djathidiro/ton...
3,How many boards Could the Mongols hoard If the...,[HH AW1] [M EH1 N IY0] [B AO1 R D Z] [K UH1 D]...,mongol hordes got bored,https://www.kaggle.com/datasets/djathidiro/ton...
4,How can a clam cram in a clean cream can?,[HH AW1] [K AE1 N] [AH0] [K L AE1 M] [K R AE1 ...,clean cream,https://www.kaggle.com/datasets/djathidiro/ton...
5,Send toast to ten tense stout saints' ten tall...,[S EH1 N D] [T OW1 S T] [T UW1] [T EH1 N] [T E...,ten tense stout saints,https://www.kaggle.com/datasets/djathidiro/ton...
...,...,...,...,...
2598,Smelly Sally sells soft satin sheep sheets.,[S M EH1 L IY0] [S AE1 L IY0] [S EH1 L Z] [S A...,smelly sally sells soft satin sheep sheets,https://archive.org/details/reallyreallygros00...
2599,Silly Sally Smits sold seven short salesmen sm...,[S IH1 L IY0] [S AE1 L IY0] [S M IH1 T S] [S O...,silly sally smits sold seven short salesmen sm...,https://archive.org/details/reallyreallygros00...
2600,Do Dewey and Didi do dizzy dances daily?,[D UW1] [D UW1 IY0] [AH0 N D] [D IY1 D IY0] [D...,dizzy dances daily,https://archive.org/details/reallyreallygros00...
2601,Please place Petey Pooh's purple porta potty b...,[P L IY1 Z] [P L EY1 S] [P IY1 T IY0] [P UW1 Z...,please place petey pooh,https://archive.org/details/reallyreallygros00...


In [57]:
#Beide DataFrames "zuschneiden"
dfTL = twistList.drop(['keywords', 'source'], axis=1)

dfKaggle = kaggle.copy()

print(dfTL.shape)
print(dfKaggle.shape)

(2602, 2)
(605, 1)


In [58]:
#dfKaggle in Phoneme übersetzen

dfKaggleP = [] #<- Liste für Phonemrepräsentation
g2p = G2p()
for index, row in dfKaggle.iterrows():
    text = row['text']
    dfKaggleP.append(g2p(text))
# print(dfKaggleP)
print(len(dfKaggleP))

dfKaggle['phonemes'] = dfKaggleP

duplic = dfKaggle[~dfKaggle['text'].isin(dfTL['text'])] #<- nur die Zungenbrecher, welche noch nicht in der TwistList vorkommen




605


In [59]:
print(len(dfKaggleP))
print(dfKaggle.shape)

print(duplic.shape)
duplic

605
(605, 2)
(74, 2)


,text,phonemes
21,"To sit in solemn silence in a dull, dark dock,...","[T, UW1, , S, IH1, T, , IH0, N, , S, AA1, L..."
29,"I scream, you scream, we all scream for icecream!","[AY1, , S, K, R, IY1, M, , ,, , Y, UW1, , ..."
34,"Celibate celebrant, celibate celebrant, celiba...","[S, EH1, L, IH0, B, AH0, T, , S, EH1, L, AH0,..."
41,"What a terrible tongue twister, what a terribl...","[W, AH1, T, , AH0, , T, EH1, R, AH0, B, AH0,..."
42,When you write copy you have the right to copy...,"[W, EH1, N, , Y, UW1, , R, AY1, T, , K, AA1..."
...,...,...
561,Then step up mister and twist your tongue,"[DH, EH1, N, , S, T, EH1, P, , AH1, P, , M,..."
562,Now Kissle will whistle at busty Miss. Russell...,"[N, AW1, , K, IH1, S, AH0, L, , W, IH1, L, ..."
576,"Purple paper people, purple paper people, purp...","[P, ER1, P, AH0, L, , P, EY1, P, ER0, , P, I..."
577,De doorgaans dappere Durgerdammer drukker Dirk...,"[D, IY1, , D, AO1, R, G, AH0, N, Z, , D, AE1..."


In [60]:
#Dataframes zusammenführen

print(dfTL.shape)
print(dfTL.columns)

print(duplic.shape)
print(duplic.columns)


finDF = dfTL.append(duplic, ignore_index=True)



print(finDF.shape)

(2602, 2)
Index(['text', 'phonemes'], dtype='object')
(74, 2)
Index(['text', 'phonemes'], dtype='object')
(2676, 2)


C:\Users\Alberto\AppData\Local\Temp\ipykernel_6328\3000386190.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finDF = dfTL.append(duplic, ignore_index=True)


In [61]:
print(finDF['text'].iloc[42])

A big black bug bit a big black dog on his big black nose!


In [62]:
### SOUNDEX

soundex = Soundex()
sx_list = []

for i in range(finDF.shape[0]):
    print(i)
    word_sx_list = []
    text = finDF['text'].iloc[i].strip()
    words = text.split(' ')
    for word in words:
        if any(char.isalpha() for char in word):
            word_sx_list.append(soundex.phonetics(word)) # Anm.: Satzzeichen werden von Soundex ignoriert.
    sx_list.append(word_sx_list)

finDF['soundex'] = sx_list


0
['Peter', 'Piper', 'picked', 'a', 'peck', 'of', 'pickled', 'peppers.', 'A', 'peck', 'of', 'pickled', 'peppers', 'Peter', 'Piper', 'picked.', 'If', 'Peter', 'Piper', 'picked', 'a', 'peck', 'of', 'pickled', 'peppers,', "Where's", 'the', 'peck', 'of', 'pickled', 'peppers', 'Peter', 'Piper', 'picked?']
1
['I', 'saw', 'Susie', 'sitting', 'in', 'a', 'shoe', 'shine', 'shop.', 'Where', 'she', 'sits', 'she', 'shines,', 'and', 'where', 'she', 'shines', 'she', 'sits.']
2
['How', 'many', 'boards', 'Could', 'the', 'Mongols', 'hoard', 'If', 'the', 'Mongol', 'hordes', 'got', 'bored?']
3
['How', 'can', 'a', 'clam', 'cram', 'in', 'a', 'clean', 'cream', 'can?']
4
['Send', 'toast', 'to', 'ten', 'tense', 'stout', "saints'", 'ten', 'tall', 'tents.']
5
['Denise', 'sees', 'the', 'fleece,', 'Denise', 'sees', 'the', 'fleas.', 'At', 'least', 'Denise', 'could', 'sneeze', 'and', 'feed', 'and', 'freeze', 'the', 'fleas.']
6
['Coy', 'knows', 'pseudonoise', 'codes.']
7
['Sheena', 'leads,', 'Sheila', 'needs.']
8
['T


['Swatch', 'watch.']
181
['Dr.', 'Johnson', 'and', 'Mr.', 'Johnson,', 'after', 'great', 'consideration,', 'came', 'to', 'the', 'conclusion', 'that', 'the', 'Indian', 'nation', 'beyond', 'the', 'Indian', 'Ocean', 'is', 'back', 'in', 'education', 'because', 'the', 'chief', 'occupation', 'is', 'cultivation.']
182
['Round', 'and', 'round', 'the', 'rugged', 'rock', 'the', 'ragged', 'rascal', 'ran.']
183
['Buckets', 'of', 'bug', 'blood,', 'buckets', 'of', 'bug', 'blood,', 'buckets', 'of', 'bug', 'blood']
184
["I'm", 'a', 'sock', 'cutter', 'and', 'I', 'cut', 'socks.', "I'm", 'a', 'sock', 'cutter', 'and', 'I', 'cut', 'socks.', "I'm", 'a', 'sock', 'cutter', 'and', 'I', 'cut', 'socks.']
185
['If', 'coloured', 'caterpillars', 'could', 'change', 'their', 'colours', 'constantly', 'could', 'they', 'keep', 'their', 'coloured', 'coat', 'coloured', 'properly?']
186
['We', 'won,', 'we', 'won,', 'we', 'won,', 'we', 'won.']
187
['Thirty-three', 'thousand', 'people', 'think', 'that', 'Thursday', 'is', 'th

In [63]:
finDF

,text,phonemes,soundex
0,Peter Piper picked a peck of pickled peppers. ...,[P IY1 T ER0] [P AY1 P ER0] [P IH1 K T] [AH0] ...,"[P360, P160, P230, A000, P200, O100, P243, P16..."
1,I saw Susie sitting in a shoe shine shop. Wher...,[AY1] [S AO1] [S UW1 Z IY0] [S IH1 T IH0 NG] [...,"[I000, S000, S200, S352, I500, A000, S000, S50..."
2,How many boards Could the Mongols hoard If the...,[HH AW1] [M EH1 N IY0] [B AO1 R D Z] [K UH1 D]...,"[H000, M500, B632, C430, T000, M524, H630, I10..."
3,How can a clam cram in a clean cream can?,[HH AW1] [K AE1 N] [AH0] [K L AE1 M] [K R AE1 ...,"[H000, C500, A000, C450, C650, I500, A000, C45..."
4,Send toast to ten tense stout saints' ten tall...,[S EH1 N D] [T OW1 S T] [T UW1] [T EH1 N] [T E...,"[S530, T230, T000, T500, T520, S330, S532, T50..."
...,...,...,...
2671,Then step up mister and twist your tongue,"[DH, EH1, N, , S, T, EH1, P, , AH1, P, , M,...","[T500, S310, U100, M236, A530, T230, Y600, T520]"
2672,Now Kissle will whistle at busty Miss. Russell...,"[N, AW1, , K, IH1, S, AH0, L, , W, IH1, L, ...","[N000, K240, W400, W234, A300, B230, M200, R24..."
2673,"Purple paper people, purple paper people, purp...","[P, ER1, P, AH0, L, , P, EY1, P, ER0, , P, I...","[P614, P160, P140, P614, P160, P140, P614, P16..."
2674,De doorgaans dappere Durgerdammer drukker Dirk...,"[D, IY1, , D, AO1, R, G, AH0, N, Z, , D, AE1...","[D000, D625, D160, D626, D626, D620, D636, D00..."


In [64]:
#Satzzeichen aus Phonemen entfernen
#finDF = finDF[1].str.replace(r'[\[!]]','',regex=True)
#print(finDF)

In [66]:
# DataFrame als .csv exportieren

# date = datetime.now().strftime('%Y-%m-%d')
# filename = f'{date}_Dataset.csv'
filename = f'tt_dataset.csv'
finDF.to_csv(filename, index= False)